# Hybrid Model using game tags

In [1]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestCentroid
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
import pandas as pd

First only content based filtering

In [2]:
#get the data 
csvPath = "../Datasets/FinalRatings.csv"

In [3]:
df = pd.read_csv (csvPath)
df = df.drop_duplicates(['user_id', 'name'], keep="last")
df = df.reset_index(drop=True)

### Select a random user

In [4]:
#select a random user that has played more that minHours #JUST LIKE LAB1
minHours = 500
df['hours'] = df['hours']
df['hours'] = df['hours'].astype(float)
users_ratings = df.groupby('user_id').hours.agg(['sum']).reindex(df.user_id).reset_index()
# users_ratings = df.groupby('user_id').total.agg(['sum']).reindex(df.idx).reset_index()
display(users_ratings)
selected = users_ratings['sum'] > minHours

selected_users = df.loc[selected]

# random_selected = selected_users.sample() 
# select_column_df = random_selected.reset_index()['user_id'] 
# selected_user = select_column_df.iloc[0] 
# print("Selected user: " + str(selected_user))

,user_id,sum
0,151603712,134.5
1,87445402,87.7
2,25096601,208.0
3,211925330,848.0
4,115396529,365.7
...,...,...
32626,154230723,923.1
32627,116564064,489.0
32628,135400225,1205.5
32629,135400225,1205.5


### Rated games by the user

In [5]:
def get_rated_games_user(selected_user):
    selected_user_ratings = df.loc[df['user_id'] == selected_user]
    rated_games_df = selected_user_ratings
    rated_games_df = rated_games_df[['name', 'game_description', 'Rating M1', 'Rating M2']]
    return rated_games_df


### Unrated games by the user

In [6]:
def get_unrated_games_user(rated_games_df):
    diff = set(df.index) - set(rated_games_df.index)
    unrated_games_df = df.loc[diff]
    unrated_games_df = unrated_games_df[['name', 'game_description']]
    unrated_games_df = unrated_games_df.drop_duplicates(subset ="name") #Had to remove duplicates 
    return unrated_games_df

## Content Based Filtering
#### Remove users who dont have at least 30 games rated

In [7]:
import numpy as np
users = np.array(selected_users['user_id'])
unique, counts = np.unique(users, return_counts=True)
main_users =list()
for i, val in enumerate(unique):
    if counts[i] >= 10:
        main_users.append(val)

#### Train KNN for each user using Rating M1 and Rating M2 seperately for all users

In [8]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestCentroid
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
allUsersList = []
pd.options.mode.chained_assignment = None  # default='warn'
# initialise the vectorizer
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
# vectorizer = CountVectorizer()
full_predictions = pd.DataFrame()
# for each gamer in the selected user list (min gameplay time of 300)
for gamer in main_users:
    rated_games_df = get_rated_games_user(gamer)
#     display(rated_games_df)
    unrated_games_df = get_unrated_games_user(rated_games_df)
        
    try:
        X = vectorizer.fit_transform(rated_games_df['game_description'])
        y = rated_games_df['Rating M1']
        indices = np.arange(X.shape[0])
        (
            X_train,
            X_test,
            y_train,
            y_test,
            indices_train,
            indices_test,
        ) = train_test_split(X, y, indices, test_size=0.3, random_state=101)

        linReg = LinearRegression().fit(X_train, y_train)
        neigh = KNeighborsRegressor(n_neighbors=10).fit(X_train, y_train)
        nearCentroid = NearestCentroid().fit(X_train, y_train)
        lr = LogisticRegression().fit(X_train, y_train)
        per = Perceptron().fit(X_train, y_train)
    
        X_unrated = vectorizer.transform(unrated_games_df['game_description'].values.astype('U'))

        # predicting already existing rated values 
        y_pred1_reg = linReg.predict(X_test)
        y_pred1_neigh = neigh.predict(X_test)
        y_pred1_nearCentroid = nearCentroid.predict(X_test)
        y_pred1_lr = lr.predict(X_test)
        y_pred1_per = per.predict(X_test)


        # predicting unrated games
        y_pred1_unrated_reg = linReg.predict(X_unrated)
        y_pred1_unrated_neigh = neigh.predict(X_unrated)
        y_pred1_unrated_nearCentroid = nearCentroid.predict(X_unrated)
        y_pred1_unrated_lr = lr.predict(X_unrated)
        y_pred1_unrated_per = per.predict(X_unrated)


        unrated_games_df['predicted_ratings_KNN M1'] = y_pred1_unrated_neigh
        unrated_games_df['predicted_ratings_Reg M1'] = y_pred1_unrated_reg
        unrated_games_df['predicted_ratings_nearCent M1'] = y_pred1_unrated_nearCentroid
        unrated_games_df['predicted_ratings_lr M1'] = y_pred1_unrated_lr
        unrated_games_df['predicted_ratings_per M1'] = y_pred1_unrated_per


        tt = rated_games_df.iloc[indices_test]

        tt['y_test_KNN M1'] = y_pred1_neigh
        tt['y_test_Reg M1'] = y_pred1_reg
        tt['y_test_nearCent M1'] = y_pred1_nearCentroid
        tt['y_test_lr M1'] = y_pred1_lr
        tt['y_test_per M1'] = y_pred1_per


        #                                                   ''' Second Algorithms '''

        y2 = rated_games_df['Rating M2']
        indices2 = np.arange(X.shape[0])
        (
            X_train2,
            X_test2,
            y_train2,
            y_test2,
            indices_train2,
            indices_test2,
        ) = train_test_split(X, y2, indices2, test_size=0.3, random_state=101)

        linReg2 = LinearRegression().fit(X_train, y_train2)
        neigh2 = KNeighborsRegressor(n_neighbors=10).fit(X_train, y_train2)
        nearCentroid2 = NearestCentroid().fit(X_train, y_train2)
        lr2 = LogisticRegression().fit(X_train, y_train2)
        per2 = Perceptron().fit(X_train, y_train2)

        y_unrated2_Reg = linReg2.predict(X_unrated)
        y_pred2_Reg = linReg2.predict(X_test)
    
        y_unrated2_neigh = neigh2.predict(X_unrated)
        y_pred2_neigh = neigh2.predict(X_test)

        y_unrated2_nearCentroid = nearCentroid2.predict(X_unrated)
        y_pred2_nearCentroid = nearCentroid2.predict(X_test)

        y_unrated2_lr = lr2.predict(X_unrated)
        y_pred2_lr = lr2.predict(X_test)

        y_unrated2_per = per2.predict(X_unrated)
        y_pred2_per = per2.predict(X_test)

        unrated_games_df['predicted_ratings_KNN M2'] = y_unrated2_neigh
        unrated_games_df['predicted_ratings_Reg M2'] = y_unrated2_Reg
        unrated_games_df['predicted_ratings_nearCent M2'] = y_unrated2_nearCentroid
        unrated_games_df['predicted_ratings_lr M2'] = y_unrated2_lr
        unrated_games_df['predicted_ratings_per M2'] = y_unrated2_per

        
        unrated_games_df_KNN_ordering = unrated_games_df.sort_values(by='predicted_ratings_KNN M2', ascending=False)
        unrated_games_df_Reg_ordering = unrated_games_df.sort_values(by='predicted_ratings_Reg M2', ascending=False)
        unrated_games_df_nearCent_ordering = unrated_games_df.sort_values(by='predicted_ratings_nearCent M2', ascending=False)
        unrated_games_df_lr_ordering = unrated_games_df.sort_values(by='predicted_ratings_lr M2', ascending=False)
        unrated_games_df_per_ordering = unrated_games_df.sort_values(by='predicted_ratings_per M2', ascending=False)
        

    #     display(unrated_games_df_KNN_ordering.head(10))

        tt['y_test_KNN M2'] = y_pred2_neigh
        tt['y_test_Reg M2'] = y_pred2_Reg
        tt['y_test_nearCent M2'] = y_pred2_nearCentroid
        tt['y_test_lr M2'] = y_pred2_lr
        tt['y_test_per M2'] = y_pred2_per

        full_predictions = full_predictions.append(tt)
        
        
    except:
        continue

In [9]:
display(full_predictions)

,name,game_description,Rating M1,Rating M2,y_test_KNN M1,y_test_Reg M1,y_test_nearCent M1,y_test_lr M1,y_test_per M1,y_test_KNN M2,y_test_Reg M2,y_test_nearCent M2,y_test_lr M2,y_test_per M2
20684,The Walking Dead,About This Series When the world is ravaged b...,4,5,1.9,1.852272,1,1,2,2.4,2.552343,1,1,5
5084,Dota 2,About This Game The most-played game on Stea...,1,1,2.3,1.921009,1,1,2,2.6,2.467740,1,1,3
20760,The Testament of Sherlock Holmes,"About This Game YOU ARE SHERLOCK HOLMES, AND...",1,1,1.7,1.754736,1,1,1,2.6,2.568771,1,1,5
21039,Surgeon Simulator,About This Game Malpractice Makes Perfect. D...,1,2,1.7,1.752004,1,1,1,2.0,2.425019,1,1,1
20965,"Papers, Please",About This Game Congratulations. The October...,1,1,1.2,1.768093,1,1,1,1.9,2.456534,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28301,Stranded Deep,About This Game Take the role of a plane cras...,3,5,2.9,3.090481,5,5,1,3.4,3.540153,5,5,5
6175,Dota 2,About This Game The most-played game on Stea...,1,1,2.9,3.141188,5,5,5,3.4,3.546267,5,5,5
15438,Survarium,About This Game Join SURVARIUM Beta Now and ...,3,5,2.9,3.018844,5,5,1,3.4,3.462722,5,5,1
26043,Besiege,About This Game Besiege is a physics based bu...,5,5,2.9,3.114093,5,5,5,3.4,3.553784,5,5,5


In [10]:
from sklearn.metrics import mean_squared_error

rmse_M1_KNN = mean_squared_error(tt['Rating M1'], tt['y_test_KNN M1'])
rmse_M2_KNN = mean_squared_error(tt['Rating M2'], tt['y_test_KNN M2'])

rmse_M1_Reg = mean_squared_error(tt['Rating M1'], tt['y_test_Reg M1'])
rmse_M2_Reg = mean_squared_error(tt['Rating M2'], tt['y_test_Reg M2'])

rmse_M1_nearCent = mean_squared_error(tt['Rating M1'], tt['y_test_nearCent M1'])
rmse_M2_nearCent = mean_squared_error(tt['Rating M2'], tt['y_test_nearCent M2'])

rmse_M1_lr = mean_squared_error(tt['Rating M1'], tt['y_test_lr M1'])
rmse_M2_lr = mean_squared_error(tt['Rating M2'], tt['y_test_lr M2'])

rmse_M1_per = mean_squared_error(tt['Rating M1'], tt['y_test_per M1'])
rmse_M2_per = mean_squared_error(tt['Rating M2'], tt['y_test_per M2'])

print('RMSE for KNN M1: ' + str(rmse_M1_KNN))
print('RMSE for KNN M2: ' + str(rmse_M2_KNN))

print('RMSE for Reg M1: ' + str(rmse_M1_Reg))
print('RMSE for Reg M2: ' + str(rmse_M2_Reg))

print('RMSE for nearCent M1: ' + str(rmse_M1_nearCent))
print('RMSE for nearCent M2: ' + str(rmse_M2_nearCent))

print('RMSE for lr M1: ' + str(rmse_M1_lr))
print('RMSE for lr M2: ' + str(rmse_M2_lr))

print('RMSE for per M1: ' + str(rmse_M1_per))
print('RMSE for per M2: ' + str(rmse_M2_per))

RMSE for KNN M1: 1.7700000000000002
RMSE for KNN M2: 3.2
RMSE for Reg M1: 1.8350931359799258
RMSE for Reg M2: 3.0808249420364353
RMSE for nearCent M1: 6.6
RMSE for nearCent M2: 3.2
RMSE for lr M1: 6.6
RMSE for lr M2: 3.2
RMSE for per M1: 5.0
RMSE for per M2: 9.6


## Collaborative Filtering 
#### UserUser with Lenskit

In [11]:
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
ratings_df1 = df[['user_id','name','Rating M1']]
ratings_df1 = ratings_df1.rename(columns={'user_id':'user', 'name': 'item', 'Rating M1': 'rating'})
user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
user_user.fit(ratings_df1)

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Numba is using threading layer omp - consider TBB
BLAS using multiple threads - can cause oversubscription
found 2 potential runtime problems - see https://boi.st/lkpy-perf


# Group Recommendations

Select random users to form a group

In [12]:
import random
group1 = random.choices(main_users, k = 5)
print(group1)

[62990992, 216785107, 162214363, 101687527, 58761868]


Get the games that they have been rated

In [13]:
#rated Games
groupdf = pd.DataFrame()
for val in group1:
    groupdf = groupdf.append(df.loc[df["user_id"] == val])
# display(groupdf)

Get the games that none of the group members rated

In [14]:
games = set(list(df[["name"]]))
unrated_groupdf = get_unrated_games_user(groupdf)
# (unrated_groupdf)

For each user in the group, train the model and then predict for the unrated games

In [15]:

vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
predDict = dict()

for gamer in group1:
    #Training
    groupdf = get_rated_games_user(gamer)
    X = vectorizer.fit_transform(groupdf['game_description'])
    y = groupdf['Rating M1']
    
    indices = np.arange(X.shape[0])
    (
        X_train,
        X_test,
        y_train,
        y_test,
        indices_train,
        indices_test,
    ) = train_test_split(X, y, indices, test_size=0.3, random_state=101)
    
    linReg = LinearRegression().fit(X_train, y_train)
#     neigh = KNeighborsRegressor(n_neighbors=10).fit(X_train, y_train)
    nearCentroid = NearestCentroid().fit(X_train, y_train)
    lr = LogisticRegression().fit(X_train, y_train)
    per = Perceptron().fit(X_train, y_train)
    
    #Predicting
    X_unrated = vectorizer.transform(unrated_groupdf['game_description'].values.astype('U'))
    y_pred1_unrated_reg = linReg.predict(X_unrated)
#     y_pred1_unrated_neigh = neigh.predict(X_unrated)
    y_pred1_unrated_nearCentroid = nearCentroid.predict(X_unrated)
    y_pred1_unrated_lr = lr.predict(X_unrated)
    y_pred1_unrated_per = per.predict(X_unrated)
    
    #add them to dictionary
    if group1.index(gamer)==0:
        predDict['reg'] = [y_pred1_unrated_reg]
#         predDict['neigh'] = [y_pred1_unrated_neigh]
        predDict['nearCentroid'] = [y_pred1_unrated_nearCentroid]
        predDict['lr'] = [y_pred1_unrated_lr]
        predDict['per'] = [y_pred1_unrated_per]
    else:
        predDict['reg'].append(y_pred1_unrated_reg)
#         predDict['neigh'].append(y_pred1_unrated_neigh)
        predDict['nearCentroid'].append(y_pred1_unrated_nearCentroid)
        predDict['lr'].append(y_pred1_unrated_lr)
        predDict['per'].append(y_pred1_unrated_per)
        


Create a dataframe for the content based ratings 

In [16]:
gameDict = dict()
for user in group1:
    userInd = group1.index(user)
    for game in unrated_groupdf['name']:
        index = list(unrated_groupdf['name']).index(game)
        gameRating = predDict['reg'][userInd][index]
        if game in gameDict.keys():
            gameDict[game].append(gameRating)
        else:
            gameDict[game] = [gameRating]
    

In [17]:
#Only the linear regression
data = (gameDict)
dfContent = pd.DataFrame(data)
dfContent.insert(loc=0, column='user_id', value=group1)


Collaborative Filtering for each user and predict for the unrated games

In [18]:
colabPredDict = dict()
for gamer in group1:
    selected_games_useruser1 = user_user.predict_for_user(gamer, list(unrated_groupdf['name']))
    print
    for item in list(unrated_groupdf['name']):
        index = list(unrated_groupdf['name']).index(item)
        gameRating = selected_games_useruser1[index] 
        if item in colabPredDict.keys():
            colabPredDict[item].append(gameRating)
        else:
            colabPredDict[item] = [gameRating]

Create a dataframe for the collaborative filtering ratings 

In [19]:
#Only the linear regression
data = (colabPredDict)
dfCollab = pd.DataFrame(data)
dfCollab.insert(loc=0, column='user_id', value=group1)
dfCollab = dfCollab.fillna(0)

In [20]:
unratedGames = list(unrated_groupdf['name'])

In [21]:
hybridDict = dict()
for game in unratedGames:
    for userRating in dfCollab[game]:
        ind = list(dfCollab[game]).index(userRating)
        collabScore = userRating
        contentScore = dfContent[game][ind]
        gameRating = (0.5*collabScore) + (0.5*contentScore)
        if game in hybridDict.keys():
            hybridDict[game].append(gameRating)
        else:
            hybridDict[game] = [gameRating]

In [22]:
data = (hybridDict)
dfHybrid = pd.DataFrame(data)
dfHybrid.insert(loc=0, column='user_id', value=group1)
dfHybrid = dfHybrid.fillna(0)

Print them all under each other so we can see whats happening 

In [23]:
display(dfCollab)
display(dfContent)
display(dfHybrid)

,user_id,Fallout 4,Left 4 Dead 2,HuniePop,Path of Exile,Poly Bridge,Left 4 Dead,Team Fortress 2,Tomb Raider,The Banner Saga,...,Kuros,Back to Bed,Legend of Fae,DinerTown Tycoon,The Impossible Game,Khet 2.0,SAMURAI WARRIORS 4-II,Life is Hard,Executive Assault,MirrorMoon EP
0,62990992,4.689189,4.353787,4.176515,2.185800,0.0,3.115702,5.096690,0.0,2.596762,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,216785107,0.000000,3.764053,4.911528,0.000000,0.0,0.000000,1.580390,0.0,2.755537,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,162214363,2.072154,1.260803,3.573961,0.000000,0.0,0.000000,1.065010,0.0,1.479720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101687527,4.716475,3.640605,5.006982,2.774378,0.0,3.113017,3.714424,0.0,2.685259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,58761868,4.989449,3.044386,3.730090,3.236169,0.0,2.641900,2.742305,0.0,2.710865,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,user_id,Fallout 4,Left 4 Dead 2,HuniePop,Path of Exile,Poly Bridge,Left 4 Dead,Team Fortress 2,Tomb Raider,The Banner Saga,...,Kuros,Back to Bed,Legend of Fae,DinerTown Tycoon,The Impossible Game,Khet 2.0,SAMURAI WARRIORS 4-II,Life is Hard,Executive Assault,MirrorMoon EP
0,62990992,2.945626,5.000000,2.608204,3.096371,2.347069,3.146062,2.779744,2.533402,2.849831,...,2.726861,2.490748,2.321660,2.826766,2.640653,2.830318,2.661296,2.909170,2.923526,2.891753
1,216785107,3.059976,2.906826,3.022092,5.000000,3.112962,3.186006,1.000000,2.949301,2.896526,...,3.207090,3.410407,3.304021,3.068871,3.328100,3.165403,3.077335,3.071730,3.000283,3.187303
2,162214363,1.635603,1.545069,1.493379,1.465557,1.494493,1.500627,1.433981,1.690217,1.542383,...,1.499408,1.615366,1.409320,1.409200,1.450605,1.403408,1.490017,1.479493,1.461888,1.463374
3,101687527,3.699833,3.669511,3.725550,3.552866,3.586746,3.702746,3.000000,3.776294,3.671662,...,3.536401,3.809219,3.608756,3.496339,3.596117,3.732726,3.734505,3.604125,3.566719,3.667285
4,58761868,5.000000,3.534830,3.362425,3.314401,3.567459,3.411439,3.591310,3.000000,3.264068,...,3.396743,3.401238,3.272367,3.349779,3.929861,3.353025,3.182600,3.469298,3.171942,3.333416


,user_id,Fallout 4,Left 4 Dead 2,HuniePop,Path of Exile,Poly Bridge,Left 4 Dead,Team Fortress 2,Tomb Raider,The Banner Saga,...,Kuros,Back to Bed,Legend of Fae,DinerTown Tycoon,The Impossible Game,Khet 2.0,SAMURAI WARRIORS 4-II,Life is Hard,Executive Assault,MirrorMoon EP
0,62990992,3.817408,4.676893,3.392360,2.641086,1.173534,3.130882,3.938217,1.266701,2.723296,...,1.363431,1.245374,1.16083,1.413383,1.320327,1.415159,1.330648,1.454585,1.461763,1.445876
1,216785107,1.529988,3.335440,3.966810,2.500000,1.173534,1.593003,1.290195,1.266701,2.826032,...,1.363431,1.245374,1.16083,1.413383,1.320327,1.415159,1.330648,1.454585,1.461763,1.445876
2,162214363,1.853878,1.402936,2.533670,2.500000,1.173534,1.593003,1.249495,1.266701,1.511051,...,1.363431,1.245374,1.16083,1.413383,1.320327,1.415159,1.330648,1.454585,1.461763,1.445876
3,101687527,4.208154,3.655058,4.366266,3.163622,1.173534,3.407881,3.357212,1.266701,3.178460,...,1.363431,1.245374,1.16083,1.413383,1.320327,1.415159,1.330648,1.454585,1.461763,1.445876
4,58761868,4.994725,3.289608,3.546258,3.275285,1.173534,3.026670,3.166807,1.266701,2.987467,...,1.363431,1.245374,1.16083,1.413383,1.320327,1.415159,1.330648,1.454585,1.461763,1.445876


Group Aggregation

In [24]:
aggf = ['Addition', 'Least_Misery', 'Most_Pleasure', 'Least_Misery+Most_Pleasure']
AggregationResult = pd.DataFrame(index = aggf, columns= unratedGames).fillna(0).astype(float)
for game in unratedGames:
    AggregationResult.at['Addition', game] = sum(dfHybrid[game])
    AggregationResult.at['Least_Misery', game] = min(dfHybrid[game])
    AggregationResult.at['Most_Pleasure', game] = max(dfHybrid[game])
    AggregationResult.at['Least_Misery+Most_Pleasure', game] = min(dfHybrid[game])+max(dfHybrid[game])

In [25]:
display(AggregationResult)

,Fallout 4,Left 4 Dead 2,HuniePop,Path of Exile,Poly Bridge,Left 4 Dead,Team Fortress 2,Tomb Raider,The Banner Saga,BioShock Infinite,...,Kuros,Back to Bed,Legend of Fae,DinerTown Tycoon,The Impossible Game,Khet 2.0,SAMURAI WARRIORS 4-II,Life is Hard,Executive Assault,MirrorMoon EP
Addition,16.404153,16.359935,17.805363,14.079992,5.867672,12.751438,13.001927,6.333504,13.226306,13.654086,...,6.817153,6.226869,5.804151,7.066914,6.601633,7.075795,6.653239,7.272925,7.308815,7.229382
Least_Misery,1.529988,1.402936,2.533670,2.500000,1.173534,1.593003,1.249495,1.266701,1.511051,1.443616,...,1.363431,1.245374,1.160830,1.413383,1.320327,1.415159,1.330648,1.454585,1.461763,1.445876
Most_Pleasure,4.994725,4.676893,4.366266,3.275285,1.173534,3.407881,3.938217,1.266701,3.178460,3.693852,...,1.363431,1.245374,1.160830,1.413383,1.320327,1.415159,1.330648,1.454585,1.461763,1.445876
Least_Misery+Most_Pleasure,6.524713,6.079829,6.899936,5.775285,2.347069,5.000884,5.187713,2.533402,4.689511,5.137468,...,2.726861,2.490748,2.321660,2.826766,2.640653,2.830318,2.661296,2.909170,2.923526,2.891753


Sort Accordingly

In [26]:
AggregationResult = AggregationResult.sort_values(by ='Least_Misery+Most_Pleasure', axis=1, ascending=False)

In [27]:
# AggregationResult = AggregationResult.sort_values(by ='Addition', axis=1, ascending=False)

Choose top 5 games

In [28]:
top5Games  = AggregationResult.columns[:5].values.ravel()
print(top5Games)

['DayZ' 'Port Royale 3' 'HuniePop' 'Borderlands 2' 'Terraria']


In [29]:
new = dfHybrid[top5Games].copy()
new = new.set_axis(top5Games, axis=1)
display(new)

,DayZ,Port Royale 3,HuniePop,Borderlands 2,Terraria
0,3.192353,4.437127,3.392360,3.839184,4.975726
1,3.313015,3.003815,3.966810,3.568567,3.783499
2,3.733939,2.492875,2.533670,1.624252,1.591859
3,5.117747,3.210372,4.366266,3.470791,4.032623
4,3.513933,2.680206,3.546258,5.022074,3.294274


Generate Explanations for the recommended games

In [30]:
from expGenerator import ExpGenerator

exp = ExpGenerator()
exp.generateExp(new)

The majority of the group will love the game: DayZ.
Port Royale 3 is not the top choice for majority, but some will love it. Why not give it a try?
The majority of the group will love the game: HuniePop.
The majority of the group will love the game: Borderlands 2.
The majority of the group will love the game: Terraria.
